In [1]:
%load_ext autoreload
import cobra
import numpy as np
from importlib import reload
import mcs
#import traceback
#import warnings
import sys
# import ray


#ray.init(ignore_reinit_error=True)

#import mcs
#import contextlib
#import os,sys
#mcs_path = os.path.dirname(mcs.__file__)
#py_path = os.path.dirname(sys.executable)
#runtime_env = {"working_dir": ".", "py_modules": [mcs],"env_vars": {"PYTHON": py_path}}
#runtime_env = {"working_dir": ".","env_vars": {"PYTHON": py_path}}
#ray.shutdown()
#ray.init(ignore_reinit_error=True,runtime_env=runtime_env)
#with contextlib.redirect_stdout(None):


In [2]:
network = cobra.io.read_sbml_model("SmallExample.sbml")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-21


In [3]:
modules = [mcs.MCS_Module(network,"target","lin_constraints",["R4 >= 1","2 R1 >= 0.5"])]
modules.append(mcs.MCS_Module(network,"desired","lin_constraints","R3 >= 1"))
modules[0].set_model_lb_ub([r.lower_bound for r in network.reactions],[r.upper_bound for r in network.reactions])

In [6]:
%autoreload
ko_cost = {'R5': 2, 'R6': 3, 'R7': 4, 'R8': 4, 'R9': 4, 'R10': 4}
ki_cost = {'R2': 6}
maxSolutions = 50
maxCost = 50

mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=5)

mcsEnum.compute_mcs();

SyntaxError: positional argument follows keyword argument (Temp/ipykernel_6920/2155645125.py, line 9)

In [4]:
ray.shutdown()
ray.is_initialized()

False

In [16]:
import traceback
import warnings
import sys

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback
warnings.simplefilter("always")

In [11]:
#import ray
#ray.init()

@ray.remote
class timer(object):
    import time
    def __init__(self):
        self.starttime = self.time.monotonic_ns()
    def timepassed(self,x):
        self.time.sleep(0.1)
        return self.time.monotonic_ns()-self.starttime

parpool = ray.util.ActorPool([timer.remote() for _ in range(int(ray.available_resources()['CPU']))])

times = list(parpool.map(lambda a,x: a.timepassed.remote(x),range(20)))

{'__name__': 'mcs', '__doc__': None, '__package__': 'mcs', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, '__spec__': ModuleSpec(name='mcs', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, origin='c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', submodule_search_locations=['c:\\users\\phili\\documents\\python\\mcs\\mcs']), '__path__': ['c:\\users\\phili\\documents\\python\\mcs\\mcs'], '__file__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', '__cached__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__pycache__\\__init__.cpython-37.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImpor

In [8]:
import time
starttime = time.monotonic_ns()
time.sleep(0.5)
print(time.monotonic_ns()-starttime)

516000000
